In [1]:
from pymatgen.core import Structure
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

In [2]:
from mofdscribe.featurizers.chemistry._fragment import get_bb_indices
from mofdscribe.featurizers.chemistry._fragment import get_bbs_from_indices
from structuregraph_helpers.subgraph import get_subgraphs_as_molecules
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.io.cif import CifWriter
import re
from pymatgen.analysis.local_env import CutOffDictNN
import yaml

from mofgraph2vec.featurize.mof2doc import MOF2doc

with open("../src/mofgraph2vec/featurize/tuned_vesta.yml", "r", encoding="utf8") as handle:
    _VESTA_CUTOFFS = yaml.load(handle, Loader=yaml.UnsafeLoader) 

import warnings
from collections import defaultdict
from typing import List, Tuple

import networkx as nx
import numpy as np
from pymatgen.analysis.graphs import MoleculeGraph, StructureGraph
from pymatgen.core import Element, Molecule, Structure

from copy import deepcopy
from typing import Dict, List, Set

from pymatgen.analysis.graphs import StructureGraph
from pymatgen.core import Structure
from structuregraph_helpers.subgraph import get_subgraphs_as_molecules

from mofdscribe.featurizers.utils.structure_graph import get_connected_site_indices
from mofdscribe.featurizers.utils.substructures import (
    _not_relevant_structure_indices,
    get_metal_indices,
)

from structuregraph_helpers.subgraph import _get_mass

/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = torch.load("../data/meta/qmof/qmof-a2d95c3.pt")

In [3]:
cif = Structure.from_file("../data/cifs/rsm/RSM0001.cif")
sg = StructureGraph.with_local_env_strategy(cif, CutOffDictNN(cut_off_dict=_VESTA_CUTOFFS))

In [4]:
indices = get_bb_indices(sg) 

/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/structuregraph_helpers/subgraph.py:121: UserWarning: No node attributes found. Using indices as node attributes.
  warnings.warn("No node attributes found. Using indices as node attributes.")


In [19]:
from torch_geometric.data import Data

In [42]:
data = Data(
    x=['O' 'C', 'Zn'], 
    num_nodes = 3, 
    edge_index=torch.LongTensor([[1], [2]]), 
    nodes=[1], 
            )

In [21]:
torch.save(data, "./test/tmp/test.pt")

In [36]:
from torch_geometric.utils import to_networkx


In [43]:
graph = to_networkx(data)

In [45]:
for node in graph.nodes:
    if node in [2, 3]:
        print(node)

2


In [5]:
indices.keys()

dict_keys(['linker_all', 'linker_functional', 'linker_scaffold', 'linker_connecting', 'nodes'])

In [6]:
def get_subs_from_indices(structure_graph: StructureGraph, indices: Set[int]):
    graph_ = structure_graph.__copy__()
    graph_.structure = Structure.from_sites(graph_.structure.sites)
    to_delete = _not_relevant_structure_indices(graph_.structure, indices)
    assert len(structure_graph) == len(to_delete) + len(indices)
    graph_.remove_nodes(to_delete)
    return graph_


In [11]:
indices.keys()

dict_keys(['linker_all', 'linker_functional', 'linker_scaffold', 'linker_connecting', 'nodes'])

In [18]:
task = "nodes"
subgraph = get_subs_from_indices(sg, [item for sublist in indices[task] for item in sublist])
CifWriter(subgraph.structure, symprec=None, write_magmoms=False, significant_figures=8, angle_tolerance=5.0, refine_struct=True).write_file("test/tmp/%s_rsm.cif" %task)

In [34]:
_, subgraph, _, _, _ = get_bbs_from_indices(sg, [item for sublist in indices["linker_all"] for item in sublist])

/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/structuregraph_helpers/subgraph.py:121: UserWarning: No node attributes found. Using indices as node attributes.
  warnings.warn("No node attributes found. Using indices as node attributes.")


In [78]:
data_loader = MOF2doc(
    cif_path=["./"],
    wl_step=2,
    n_components=20,
    use_hash=False,
    writing_style="paragraph",
    composition=False,
)

In [79]:
data_loader.get_documents()

100%|██████████| 7/7 [00:00<00:00, 25.12it/s]


[TaggedDocument(words=['Zn_O_O_O_O', 'Zn_O_O', 'O_Zn_Zn_Zn_Zn', 'O_Zn', 'Zn_O_O_O_O_O_Zn_O_Zn_O_Zn_O_Zn_Zn_Zn_Zn', 'Zn_O_O_O_Zn_O_Zn_Zn_Zn_Zn', 'O_Zn_Zn_Zn_Zn_Zn_O_O_O_O_Zn_O_O_O_O_Zn_O_O_O_O_Zn_O_O_O_O', 'O_Zn_Zn_Zn_Zn_Zn_O_O_Zn_O_O_Zn_O_O_Zn_O_O', 'O_Zn_Zn_O_O_O_O', 'O_Zn_Zn_O_O'], tags=['linker']),
 TaggedDocument(words=['O', 'O'], tags=['linker_connecting']),
 TaggedDocument(words=['H_C', 'C_C', 'C_C_C_C', 'C_C_C_H', 'H_C_C_C_C_H', 'C_C_C_C_C_C', 'C_C_C_C_C_C_C_C_C_H_C_C_C_H', 'C_C_C_H_C_C_C_C_C_C_C_H_H_C'], tags=['linker_scaffold']),
 TaggedDocument(words=['Zn', 'Zn'], tags=['nodes']),
 TaggedDocument(words=['Zn', 'Zn'], tags=['linker_functional']),
 TaggedDocument(words=['Zn_O_O_O_O', 'H_C', 'C_C_O_O', 'C_C_C_C', 'C_C_C_H', 'O_Zn_Zn_Zn_Zn', 'O_C_Zn', 'Zn_O_O_O_O_O_C_Zn_O_C_Zn_O_C_Zn_O_Zn_Zn_Zn_Zn', 'H_C_C_C_C_H', 'C_C_O_O_C_C_C_C_O_C_Zn_O_C_Zn', 'C_C_C_C_C_C_C_H_C_C_C_H_C_C_O_O', 'C_C_C_H_C_C_C_C_C_C_C_H_H_C', 'O_Zn_Zn_Zn_Zn_Zn_O_O_O_O_Zn_O_O_O_O_Zn_O_O_O_O_Zn_O_O_O_O', 'O_C_Zn_C

In [208]:
cif.formula

'Zn8 H24 C48 O26'

In [28]:
len(sub_linker.structure.sites)

98

In [24]:
len(cif.sites)

106

In [21]:
[x for x in np.unique(py_strucs[0].species) if x.is_metal]

[Element Fe]

In [22]:
df = pd.read_csv("../data/data.csv")

In [25]:
py_strucs = [Structure.from_file("../data/cifs/rsm/%s.cif" %c) for c in tqdm(df["cif.label"])]

  0%|          | 3/3257 [00:00<02:18, 23.58it/s]/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
100%|██████████| 3257/3257 [01:37<00:00, 33.54it/s]


In [30]:
metal_list = [[x.symbol for x in np.unique(struc.species) if x.is_metal] for struc in tqdm(py_strucs)]

100%|██████████| 3257/3257 [00:08<00:00, 365.11it/s]


In [31]:
for x in metal_list:
    if len(x) != 1:
        print(x)

['Gd', 'Ag']
['Cu', 'W']
['Al', 'Ag']
['Gd', 'Co']
['Cd', 'Ni']
['Na', 'Cd']
['Cu', 'Mo']
['K', 'La']
['K', 'Ce']
['Na', 'Zn']
['K', 'Zn']
['Cd', 'Ni']
['Cu', 'W']
['Tb', 'Cu']
['Mn', 'Fe']
['K', 'Sc']
['Cu', 'Mo']
['Cu', 'W']
['Zn', 'Co']
['Ca', 'Pb']
['Mn', 'Cu']
['Li', 'Zn']
['Lu', 'W']
['Na', 'Zn']
['Rb', 'Co']
['Ba', 'Cd']
['Na', 'Co']
['Zn', 'Cu']
['Zn', 'Co']
['K', 'Ni']
['Fe', 'Ni']
['Cu', 'Mo']
['Cu', 'Mo']
['La', 'Co']
['Ni', 'W']
['Ni', 'Mo']
['La', 'Cu']
['Ca', 'Cd']
['Ni', 'Mo']
['Ho', 'Cu']
['Na', 'Tb']
['La', 'Mg']
['Mn', 'Cu']
['Mn', 'Co']
['Eu', 'W']
['Ba', 'Cu']
['Gd', 'Ag']
['Cu', 'W']
['Mn', 'Cr', 'Fe']
['Cu', 'W']
['Sr', 'Zn']
['Sr', 'Ni']
['La', 'Eu']
['Ca', 'Ce']
['Ba', 'Zn']
['Ba', 'Co']
['Cu', 'W']
['Tb', 'Eu']
['Ni', 'Mo']
['Tb', 'Cu']
['Tb', 'Cu']
['Sm', 'Zn']
['Na', 'Zn', 'Co']
['Eu', 'Ag']
['Ce', 'Mg']
['Nb', 'Cu']
['Na', 'Zn']
['Na', 'Zn', 'Fe']
['Mg', 'Cd', 'Mo']
['Mn', 'Au']
['Mn', 'Ag']
['La', 'Ni']
['K', 'Fe', 'Ni']
['Co', 'Cu']
['Sr', 'U']
['Na', 'Co'

In [32]:
df["metal_1"] = [ele[0] for ele in metal_list]

In [36]:
metal_2 = [ele[1] if len(ele)>1 else np.NaN for ele in metal_list]

In [44]:
metal_3 = [ele[2] if len(ele)>2 else np.NaN for ele in metal_list]

In [45]:
metal_4 = [ele[3] if len(ele)>3 else np.NaN for ele in metal_list]

In [43]:
df["metal_2"] = metal_2

In [46]:
df["metal_3"] = metal_3

In [47]:
df["metal_4"] = metal_4

In [49]:
df.loc[:, ~df.columns.str.match('Unnamed: 0')]

,in,ASA [m^2/cm^3],CellV [A^3],Df,Di,Dif,NASA [m^2/cm^3],POAV [cm^3/g],POAVF,PONAV [cm^3/g],...,KH_N2_molkgpa,log_selectivity,info.decorated_graph_hash,info.undecorated_graph_hash,info.decorated_scaffold_hash,info.undecorated_scaffold_hash,metal_1,metal_2,metal_3,metal_4
0,8,2163.580,20019.00,7.83744,9.47418,9.45292,0.000000,1.074720,0.64186,0.000000,...,4.160970e-06,0.449093,382517780dc5ab8eb8fa43295ce8bdf4,b49059acbd8c344ef658b5b181a142f5,25800f339ff3d43d2ddc5d1a59b4eeb4,08515bc3d51c5d80eafef0baa6e2b134,Cu,NaN,NaN,NaN
1,9,641.266,2633.04,4.14331,4.82084,4.80387,0.000000,0.136436,0.24024,0.000000,...,7.416090e-08,NaN,a84c197bbfa5810808c44dc4ccfe11af,874ad4e1f85e1de8bd54bed65199e13a,b526ee2136f2d05af53b2bed85c5339d,b02ed13751c7be6d0f820f3e5f81b9b3,La,NaN,NaN,NaN
2,10,1603.800,15518.00,13.71867,18.80790,18.79960,0.888134,0.957188,0.64550,0.000000,...,6.027810e-06,0.533550,45efa7c1e878af5ecbc618b706fc1136,bccd7095b8e6d26f8257ae674095d645,13cae2b9e4cfe2cfe9d05a9468ae4830,eb2900477384511abd851c626625231e,In,NaN,NaN,NaN
3,11,600.107,2602.59,4.06893,4.80529,4.80529,0.000000,0.133609,0.23884,0.000000,...,1.174370e-06,0.522879,67abee315179d34012433698bf59a924,9fa41a3543983b720923f5ffb080b005,57a7b96939ec88db393126c63b0ed48d,0a8666fb6118b8149daf8d5e89321c93,Ce,NaN,NaN,NaN
4,15,498.734,2493.02,3.94356,4.70447,4.68752,0.000000,0.113502,0.21898,0.000000,...,1.454260e-06,0.548749,3f69a0844ce43a2c27086353118d05ed,44a150af84481dfe02a60f89af03568c,8cd5110f581b241624b3544555208c56,9be47b63130f6c51d0ab9c1bf63e789e,Eu,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,9677,601.756,4479.64,4.08689,4.75984,4.75984,0.000000,0.163833,0.25402,0.000000,...,3.365250e-06,0.868569,df97f8d55b270ccf87d6b5ef1d240b49,b9fbb38978dc92454d5f9dadc2d1f889,b7ce948b3839045aeca55c2b2b8c5831,04093fb5811c815186f3c99d9bf510d0,Cd,NaN,NaN,NaN
3253,9679,1607.600,2134.77,7.69493,9.54888,9.54888,0.000000,0.661485,0.57334,0.000000,...,3.589710e-06,0.509364,bcd37c930e2b58a516766833377a8c77,d7f19b4984eefc1dfe81e7e5d11a2c3c,3363bfcc7baaa3e433d11644fc5fa6ee,4cd3ef34cda4fc1a43dccccec19778b1,Zn,NaN,NaN,NaN
3254,9681,416.131,1636.02,4.20115,4.73990,4.72096,0.000000,0.093092,0.16366,0.000000,...,7.976930e-06,1.153723,f5a545a2d506d8424bfbbfd2917b9b9b,006d95bcc7ab668a21894bcb66e6b445,5a0a0630fef8a2bafe333d8fe4388354,ba67a8fe5b11dba57bc563a5d3b7a543,Zn,NaN,NaN,NaN
3255,9685,0.000,2044.51,2.56035,4.18124,4.16282,35.643400,0.000000,0.00000,0.024759,...,3.393710e-06,NaN,47335f2329fe60935ca636a63a0ebd48,70e36dc2a0dd36e20a761769501aa3f9,f0900a12ad6d994e6c6a490cedc50406,55501fe2ad68169f472605c0ab6f6cd2,K,Gd,NaN,NaN


In [50]:
df.to_csv("../data/rsm_metal.csv", index=False)

In [3]:
df_emb = pd.read_csv("../experiments/workflow/wandb/offline-run-20230523_190357-93u4mjhz/tmp/embedding_dv.csv").set_index("type")

In [11]:
df_sto = pd.read_csv("../data/embedding_example/rsm-sto120.csv").set_index("type")

In [13]:
df_com = df_emb.join(df_sto)

In [14]:
df_com

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,range Electronegativity,mean Electronegativity,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons
type,,,,,,,,,,,,,,,,,,,,,
RSM0001,0.101371,0.215530,0.033499,-0.141782,0.005453,0.175517,-0.077258,-0.087504,-0.009211,0.037494,...,2.240,2.776154,1.769231,2.307692,0.076923,0.538462,0.377049,0.491803,0.016393,0.114754
RSM0002,0.170878,0.339871,0.004684,-0.179333,-0.039081,0.375669,-0.173313,-0.155345,-0.082482,0.004756,...,2.270,2.773846,1.769231,2.307692,0.000000,0.461538,0.389831,0.508475,0.000000,0.101695
RSM0003,0.130742,0.187140,0.008065,-0.247930,-0.052194,0.381775,-0.115752,-0.149821,-0.014116,0.007495,...,2.230,2.776923,1.769231,2.307692,0.000000,0.692308,0.370968,0.483871,0.000000,0.145161
RSM0004,0.130990,0.293535,-0.022066,-0.178386,0.008399,0.325342,-0.112160,-0.126959,-0.022916,-0.059256,...,2.255,2.775000,1.769231,2.307692,0.000000,0.538462,0.383333,0.500000,0.000000,0.116667
RSM0008,0.072972,0.150865,0.030419,-0.113763,0.068517,0.171237,-0.002723,0.008092,0.002610,0.049884,...,2.130,2.729091,1.909091,2.363636,0.000000,0.000000,0.446809,0.553191,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RSM4610,0.039640,0.344323,0.157458,-0.120436,-0.020549,0.705851,-0.560834,0.038037,-0.446579,0.162828,...,1.790,2.470651,1.587455,1.319662,0.349819,0.000000,0.487407,0.405185,0.107407,0.000000
RSM4613,-0.094842,0.157643,0.098664,0.049347,0.162792,0.177498,-0.316448,-0.048457,-0.102722,-0.012007,...,1.790,2.510426,1.617021,1.446809,0.212766,0.000000,0.493506,0.441558,0.064935,0.000000
RSM4614,-0.013330,0.037651,-0.021806,0.047638,0.050852,0.188703,-0.000857,-0.094424,-0.069792,0.055648,...,1.540,2.547222,1.694444,1.638889,0.277778,0.000000,0.469231,0.453846,0.076923,0.000000


In [8]:
df_com.to_csv("../data/embedding_example/rsm_sto120_mof2vec.csv")

In [77]:
df_combin = df_emb.join(df_sto_mean)

In [78]:
df_combin

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,MatscholarElementData mean embedding 191,MatscholarElementData mean embedding 192,MatscholarElementData mean embedding 193,MatscholarElementData mean embedding 194,MatscholarElementData mean embedding 195,MatscholarElementData mean embedding 196,MatscholarElementData mean embedding 197,MatscholarElementData mean embedding 198,MatscholarElementData mean embedding 199,MatscholarElementData mean embedding 200
type,,,,,,,,,,,,,,,,,,,,,
RSM0001,0.101371,0.215530,0.033499,-0.141782,0.005453,0.175517,-0.077258,-0.087504,-0.009211,0.037494,...,0.119388,-0.040717,-0.033272,-0.054816,-0.004912,0.015728,-0.003543,0.054782,-0.002767,0.024165
RSM0002,0.170878,0.339871,0.004684,-0.179333,-0.039081,0.375669,-0.173313,-0.155345,-0.082482,0.004756,...,0.121532,-0.041453,-0.034330,-0.056544,-0.004094,0.021566,-0.005489,0.056697,-0.002465,0.022273
RSM0003,0.130742,0.187140,0.008065,-0.247930,-0.052194,0.381775,-0.115752,-0.149821,-0.014116,0.007495,...,0.121614,-0.047928,-0.031848,-0.055058,-0.004176,0.014536,-0.000956,0.053780,-0.000682,0.018945
RSM0004,0.130990,0.293535,-0.022066,-0.178386,0.008399,0.325342,-0.112160,-0.126959,-0.022916,-0.059256,...,0.120118,-0.044087,-0.034217,-0.059996,-0.002523,0.021942,0.001433,0.057742,-0.005464,0.019835
RSM0008,0.072972,0.150865,0.030419,-0.113763,0.068517,0.171237,-0.002723,0.008092,0.002610,0.049884,...,0.107734,-0.036597,-0.018737,-0.057656,0.008754,-0.001519,0.019139,0.053759,-0.022151,0.026957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RSM4610,0.039640,0.344323,0.157458,-0.120436,-0.020549,0.705851,-0.560834,0.038037,-0.446579,0.162828,...,0.121005,-0.044259,-0.026141,-0.029223,-0.011680,-0.020062,0.017488,0.047380,0.007716,0.032073
RSM4613,-0.094842,0.157643,0.098664,0.049347,0.162792,0.177498,-0.316448,-0.048457,-0.102722,-0.012007,...,0.122189,-0.044788,-0.025178,-0.032462,-0.007945,-0.019158,0.019006,0.044053,0.004590,0.034380
RSM4614,-0.013330,0.037651,-0.021806,0.047638,0.050852,0.188703,-0.000857,-0.094424,-0.069792,0.055648,...,0.115618,-0.043455,-0.017459,-0.038193,-0.000930,-0.024042,0.027288,0.043123,-0.003907,0.034360


In [80]:
df_combin = df_combin.join(df_label[["logKH_CH4", "logKH_CO2"]])

In [81]:
df_train, df_test = train_test_split(df_combin, test_size=0.2)

In [82]:
df_train.to_csv("/Users/xiaoqi/Codes/candidacy/database/rsm_composition_mof2vec_train.csv")

In [83]:
df_test.to_csv("/Users/xiaoqi/Codes/candidacy/database/rsm_composition_mof2vec_test.csv")